In [6]:
from gurobipy import Model, GRB
import networkx as nx
import matplotlib.pyplot as plt

cities = {"Germany" :["Berlin", "Frankfurt"], 
          "Australia":["Melbourne", "Sydney"], 
          "The US of A":["New York", "Los Angeles"],
          "Japan":["Tokyo", "Osaka"],
          "France":["Paris", "Lyon"],
          "UK":["London", "Manchester"],
          "Canada":["Toronto", "Vancouver"],
          "Iran":["Tehran", "Shiraz"],
          "South Africa":["Cape Town", "Johannesburg"],
          "Norge":["Oslo", "Bergen"]}

countries = {
    "Germany": ["Berlin", "Frankfurt"],
    "Australia": ["Melbourne", "Sydney"],
    "USA": ["New York", "Los Angeles"],
    "Japan": ["Tokyo", "Osaka"],
    "France": ["Paris", "Lyon"],
    "UK": ["London", "Manchester"],
    "Canada": ["Toronto", "Vancouver"],
    "Iran": ["Tehran", "Shiraz"],
    "South Africa": ["Cape Town", "Johannesburg"],
    "Norway": ["Oslo", "Bergen"]
    }

n = len(city_list)


model = Model("Optimization_Model")

x = model.addVars(n, n, vtype = GRB.BINARY, name = "x")

model.setObjective(sum(x[i, j] for i in range(n) for j in range(n)))

for i in range(n):
    model.addConstr(sum(x[i,j] for j in range(n))== 1) #So that it visits exactly one city
    model.addConstr(sum(x[j,i] for j in range(n))== 1) #So exactly one city is visited
    
for country, cities_in_country in cities.items():
    city_indices = [cities.index(city) for city in country]
    
    for i in city_indices:
        for j in city_indices:
            if i != j:
                model.addConstr(x[i, j] == 0)

start = 0


model.addConstr(sum(x[start, j] for j in range(n) if j != start)==1)
model.addConstr(sum(x[i, start] for i in range(n) if i != start)==1)
    
model.optimize()

G = nx.DiGraph()
for city in cities:
    G.add_node(city)

if model.Status == GRB.OPTIMAL:
    solve = model.getAttr("x", x)
    route = [-1] * (n) # to store the route of the cities
    for i in range(n):
        for j in range(n):
            if solve[i, j] > 0.5:
                route[j] = i
                #print(f"City {i} is in position {j}") Test script
    city_order = [city_list[i] for i in route]
    
    city_order.append(city_order[0])
    print("Optimal Order:", "->".join(city_order))
    for i in range(len(city_order) - 1):
        G.add_edge(city_order[i], city_order[i + 1])
    
    pos = nx.circular_layout(G)
    nx.draw(G, pos, with_labels=True, node_size=2000, node_color="lightblue", font_size=10)
    
    plt.title("Optimale Route")
    plt.show()
else:
    print("Shit Fucked up")

AttributeError: 'dict' object has no attribute 'index'